In [1]:
import csv
import pandas as pd
from typing import Callable

In [2]:
columns = [
    "characterID", "character_name", 
    "movieID", "movie_title", "gender", 
    "position-in_credits"
]
def foo(x):
    rest = " ".join(x[4:])
    return x[:4] + [rest]
movie_characters_metadata = pd.read_csv("data/movie_characters_metadata.tsv", sep="\t", names=columns, on_bad_lines=foo, engine="python")
movie_characters_metadata.head(3)

,characterID,character_name,movieID,movie_title,gender,position-in_credits
0,u0,BIANCA,m0,10 things i hate about you,f,4
1,u1,BRUCE,m0,10 things i hate about you,?,?
2,u2,CAMERON,m0,10 things i hate about you,m,3


In [3]:
character2gender = dict(movie_characters_metadata[["character_name", "gender"]].values)

In [4]:
columns = ["movieID", "movie_title", "movie_year", "IMDB_rating", "IMDB_votes_count","movie_genres"]
movie_titles_metadata = pd.read_csv("data/movie_titles_metadata.tsv", sep="\t", names=columns)
movie_titles_metadata.head(3)

,movieID,movie_title,movie_year,IMDB_rating,IMDB_votes_count,movie_genres
0,m0,10 things i hate about you,1999,6.9,62847.0,['comedy' 'romance']
1,m1,1492: conquest of paradise,1992,6.2,10421.0,['adventure' 'biography' 'drama' 'history']
2,m2,15 minutes,2001,6.1,25854.0,['action' 'crime' 'drama' 'thriller']


In [5]:
def foo(x):
    rest = " ".join(x[4:])
    return x[:4] + [rest]
columns = ["lineID", "characterID", "movieID", "characterName", "utterance"]
movie_lines = pd.read_csv("data/movie_lines.tsv", sep="\t", names=columns, quoting=csv.QUOTE_NONE, on_bad_lines=foo, engine="python")
movie_lines["lineID"] = movie_lines["lineID"].str.lstrip('"')
movie_lines.head(3)

,lineID,characterID,movieID,characterName,utterance
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.


In [6]:
line2utterance_mapping = dict(movie_lines[["lineID", "utterance"]].values)
line2characterName_mapping = dict(movie_lines[["lineID", "characterName"]].values)

In [7]:
columns = ["first_character_id", "second_character_id", "movieID", "dialog"]
movie_conversations = pd.read_csv("data/movie_conversations.tsv", sep="\t", names=columns)
movie_conversations.head(3)

,first_character_id,second_character_id,movieID,dialog
0,u0,u2,m0,['L194' 'L195' 'L196' 'L197']
1,u0,u2,m0,['L198' 'L199']
2,u0,u2,m0,['L200' 'L201' 'L202' 'L203']


In [22]:
df = pd.merge(
    movie_conversations[["movieID", "dialog"]], 
    movie_titles_metadata[["movieID", "movie_title", "movie_genres"]], 
    on="movieID"
)
df.head(3)

,movieID,dialog,movie_title,movie_genres
0,m0,['L194' 'L195' 'L196' 'L197'],10 things i hate about you,['comedy' 'romance']
1,m0,['L198' 'L199'],10 things i hate about you,['comedy' 'romance']
2,m0,['L200' 'L201' 'L202' 'L203'],10 things i hate about you,['comedy' 'romance']


In [23]:
def foo(mapping: Callable, raw_dialog: str):
    dialog = raw_dialog.strip("[]").split()
    return [mapping[l.strip("'")] for l in dialog]
df["speakers"] = df["dialog"].apply(lambda x: foo(line2characterName_mapping, x))
df["dialog"] = df["dialog"].apply(lambda x: foo(line2utterance_mapping, x))
df["dialog_id"] = list(range(len(df)))
df.to_csv("data/dialogs.csv", index=False)
print(len(df))
df.head(3)

82961


,movieID,dialog,movie_title,movie_genres,speakers,dialog_id
0,m0,[Can we make this quick? Roxanne Korrine and ...,10 things i hate about you,['comedy' 'romance'],"[BIANCA, CAMERON, BIANCA, CAMERON]",0
1,m0,[You're asking me out. That's so cute. What's...,10 things i hate about you,['comedy' 'romance'],"[BIANCA, CAMERON]",1
2,m0,[No no it's my fault -- we didn't have a prope...,10 things i hate about you,['comedy' 'romance'],"[BIANCA, CAMERON, BIANCA, CAMERON]",2


In [24]:
df = df.explode(["dialog", "speakers"])
df = df[["movieID", "dialog_id", "dialog", "speakers", "movie_title", "movie_genres"]]
df.head(3)

,movieID,dialog_id,dialog,speakers,movie_title,movie_genres
0,m0,0,Can we make this quick? Roxanne Korrine and A...,BIANCA,10 things i hate about you,['comedy' 'romance']
0,m0,0,Well I thought we'd start with pronunciation i...,CAMERON,10 things i hate about you,['comedy' 'romance']
0,m0,0,Not the hacking and gagging and spitting part....,BIANCA,10 things i hate about you,['comedy' 'romance']


In [25]:
df.to_csv("data/dialogs.csv", index=False)